In [1]:
import pandas as pd
import numpy as np
import mysql.connector

### 建立甄選Data到DB

In [2]:
dfRegular = pd.read_excel("./文件集/112四技二專甄選_一般組.xls")
dfRegular['招生群（類）別']

0             01 機械群
1           02 動力機械群
2             01 機械群
3       03 電機與電子群電機類
4             05 化工群
            ...     
2743        02 動力機械群
2744          14 農業群
2745          17 餐旅群
2746          07 設計群
2747       20 藝術群影視類
Name: 招生群（類）別, Length: 2748, dtype: object

In [9]:
recom_adm_data = []
classes = np.sort(dfRegular['招生群（類）別'].unique())
for group in classes:
    recom_adm_data.append(tuple(group.split()))
recom_adm_data

[('01', '機械群'),
 ('02', '動力機械群'),
 ('03', '電機與電子群電機類'),
 ('04', '電機與電子群資電類'),
 ('05', '化工群'),
 ('06', '土木與建築群'),
 ('07', '設計群'),
 ('08', '工程與管理類'),
 ('09', '商業與管理群'),
 ('10', '衛生與護理類'),
 ('11', '食品群'),
 ('12', '家政群幼保類'),
 ('13', '家政群生活應用類'),
 ('14', '農業群'),
 ('15', '外語群英語類'),
 ('16', '外語群日語類'),
 ('17', '餐旅群'),
 ('18', '海事群'),
 ('19', '水產群'),
 ('20', '藝術群影視類'),
 ('21', '資管類')]

In [12]:
conn = mysql.connector.connect(user='root', password='root', host='localhost', database='四技二專')
cursor = conn.cursor()
sql = "insert into 甄選 values(%s, %s, null)"
cursor.executemany(sql, recom_adm_data)
conn.commit()

In [13]:
cursor.close()
conn.close()